In [ ]:
!pip install datasets transformers
!pip install evaluate
!apt install git-lfs
!pip install datasets

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [5]:
import transformers
import json
from collections import namedtuple
from random import shuffle
import datasets
import pandas as pd
import evaluate
from tqdm import tqdm 
from datasets import load_dataset, load_metric

# Fine-tuning a model on a question-answering task

In [7]:
squad_v2 = False
model_checkpoint = "xlm-roberta-base"
batch_size = 16

## Loading the dataset

In [8]:
def load_question_agument():
    qs = []
    with open(f"/content/drive/MyDrive/Colab/UIT_vQuaD/augument.txt",'r') as f:
        for line in f:
          qs.append(line[:-1])
    return qs 
qs = load_question_agument()
qs[:3]

['Phạm Văn Đồng đã sử dụng tên gì khi còn là phó giám đốc Sở Tư pháp ở Quế Lâm?',
 'Phạm Văn Đồng giữ chức vụ gì trong bộ máy nhà nước CHXHCNVN?',
 'Từ 1955 đến 1976, Phạm Văn Đồng giữ chức vụ gì?']

In [9]:
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 118 kB 36.2 MB/s 


In [10]:
bleu = evaluate.load("sacrebleu")
def bleu_2_sent(s,t):
  results = bleu.compute(predictions=[s], references=[[t]])
  # print(results)
  return results['score']

bleu_2_sent('toi la chieu ddd','toi la chieu a b c')

36.06452879987789

In [16]:

def load_train_data(name):

  data = []

  dirs = ["/content/drive/MyDrive/Colab/UIT_vQuaD/dev-context-en-question-vi.json", 
          "/content/drive/MyDrive/Colab/UIT_vQuaD/dev-context-vi-question-en.json",
          "/content/drive/MyDrive/Colab/UIT_vQuaD/test-context-en-question-vi.json",
          "/content/drive/MyDrive/Colab/UIT_vQuaD/test-context-vi-question-en.json"
          ]
  for dir in dirs :
      with open(dir,'r') as f:
           tmp  = json.load(f)
           for i in tqdm(range(len(tmp['data']))):
                o = tmp['data'][i]
                title = tmp['data'][i]['title']
                n = len(tmp['data'][i]['paragraphs'])
                for j in range(n):
                    context = tmp['data'][i]['paragraphs'][j]['context']
                    qas = tmp['data'][i]['paragraphs'][j]['qas']
                    for qa in qas:
                        question = qa['question']
                        id = qa['id'] 
                        # print(qa['answers'])
                        if len(qa['answers']) == 0:
                            continue
                        answer_start = qa['answers'][0]['answer_start']
                        answer_text = qa['answers'][0]['text']
                        
                        data.append({
                                'id':id,
                                'title':title,
                                'context': context,
                                'question': question ,
                                'answers':{
                                    'text':[answer_text],
                                    'answer_start':[answer_start]
                                }
                            })

  tmp = []
  with open(f"/content/drive/MyDrive/Colab/UIT_vQuaD/{name}.json",'r') as f:
       tmp  = json.load(f)

  
  cnt = 0 
  for i in tqdm(range(len(tmp['data']))):
      o = tmp['data'][i]
      title = tmp['data'][i]['title']
      n = len(tmp['data'][i]['paragraphs'])

      for j in range(n):
          context = tmp['data'][i]['paragraphs'][j]['context']
          qas = tmp['data'][i]['paragraphs'][j]['qas']
          for qa in qas:
              question = qa['question']
              id = qa['id'] 
              # print(qa['answers'])
              if len(qa['answers']) == 0:
                  continue
              answer_start = qa['answers'][0]['answer_start']
              answer_text = qa['answers'][0]['text']
              
              data.append({
                      'id':id,
                      'title':title,
                      'context': context,
                      'question': question ,
                      'answers':{
                          'text':[answer_text],
                          'answer_start':[answer_start]
                      }
                  })
              
              # agument data 
              bleu_score = bleu_2_sent(qs[cnt],question)
              if 15 <= bleu_score and bleu_score <= 95:
                data.append({
                        'id':id,
                        'title':title,
                        'context': context,
                        'question': qs[cnt] ,
                        'answers':{
                            'text':[answer_text],
                            'answer_start':[answer_start]
                        }
                    })
              cnt += 1
  tdata = pd.DataFrame(data)
  tdata = datasets.Dataset.from_pandas(tdata)
  return tdata

data_train = load_train_data('train')
# data_test = load_train_data('test')

100%|██████████| 138/138 [01:04<00:00,  2.15it/s]


In [17]:
data_train

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 44326
})

In [18]:
def load_test_data(name):
  with open(f"/content/drive/MyDrive/Colab/UIT_vQuaD/{name}.json",'r') as f:
       tmp  = json.load(f)
  data = []
  for i in range(len(tmp['data'])):
      o = tmp['data'][i]
      title = tmp['data'][i]['title']
      n = len(tmp['data'][i]['paragraphs'])

      for j in range(n):
          context = tmp['data'][i]['paragraphs'][j]['context']
          qas = tmp['data'][i]['paragraphs'][j]['qas']
          for qa in qas:
              question = qa['question']
              id = qa['id'] 
              # print(qa['answers'])
              if len(qa['answers']) == 0:
                  continue
              answer_start = []
              answer_text = []
              for t in range(len(qa['answers'])):
                  answer_start.append(qa['answers'][t]['answer_start'])
                  answer_text.append(qa['answers'][t]['text'])
              data.append({
                  'id':id,
                  'title':title,
                  'context': context,
                  'question': question ,
                  'answers':{
                          'text':answer_text,
                          'answer_start':answer_start
                      }
                  
              })
  tdata = pd.DataFrame(data)
  tdata = datasets.Dataset.from_pandas(tdata)
  return tdata

data_test = load_test_data('test')


In [ ]:
data_train = data_train.shuffle(seed=42)

In [19]:
DATA = namedtuple('data', ['train', 'test',])
data = DATA(data_train,data_test)

## Preprocessing the training data

In [20]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [21]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
pad_on_right = tokenizer.padding_side == "right"

In [22]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

tokenized_train = data.train.map(prepare_train_features, batched=True, remove_columns=data.train.column_names)
tokenized_test = data.test.map(prepare_train_features, batched=True, remove_columns=data.test.column_names)

  0%|          | 0/45 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [23]:
data

data(train=Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 44326
}), test=Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 2210
}))

In [24]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples
tokenized_test_prepared = data.test.map(prepare_validation_features, batched=True, remove_columns=data.test.column_names)


  0%|          | 0/3 [00:00<?, ?ba/s]

In [25]:
tokenized_train

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 50611
})

## Fine-tuning the model

In [26]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
model = AutoModelForQuestionAnswering.from_pretrained("xlm-roberta-base")

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForQuestionAnswering: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream tas

In [27]:
import evaluate
metric = evaluate.load("squad")

In [28]:
import collections 
import numpy as np
from tqdm.auto import tqdm

n_best=20 
max_answer_length = 30
examples = data.test
features = tokenized_test_prepared

def compute_metrics(evals):
    (start_logits, end_logits) , _ = evals
    # print(start_logits)
    # print('-'*100)
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

Then we will need a data collator that will batch our processed examples together, here the default one will work:

In [29]:
total = len(data.train)
batchsz = 16 
st = int(total / 64)
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"./{model_name}-finetuned-squad-4",
    evaluation_strategy = "steps",
    logging_strategy = "steps",
    eval_steps=st,
    logging_steps=st,
    save_steps=st,
    learning_rate=2e-5,
    per_device_train_batch_size=batchsz,
    per_device_eval_batch_size=batchsz,
    gradient_accumulation_steps=1,
    num_train_epochs=6,
    weight_decay=0.01,
    # logging_steps=2,
    metric_for_best_model='f1',
    save_total_limit=10,
    push_to_hub=True,
)

In [ ]:
args

In [30]:
from transformers import default_data_collator
data_collator = default_data_collator

In [ ]:
import gc
gc.collect()
trainer = None

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

/usr/local/lib/python3.7/dist-packages/huggingface_hub/repository.py:729: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.12. Please create the repository first using `create_repo(..., exists_ok=True)`.
  FutureWarning,
Cloning https://huggingface.co/chieunq/xlm-roberta-base-finetuned-squad-4 into local empty directory.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 50611
  Num Epochs = 6
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 18984
  Number of trainable parameters = 277454594


Step,Training Loss,Validation Loss,Exact Match,F1
692,2.830200,1.767404,48.371041,69.385328
1384,1.750100,1.443659,56.968326,75.988236


***** Running Evaluation *****
  Num examples = 2359
  Batch size = 16


  0%|          | 0/2210 [00:00<?, ?it/s]

Saving model checkpoint to ./xlm-roberta-base-finetuned-squad-4/checkpoint-692
Configuration saved in ./xlm-roberta-base-finetuned-squad-4/checkpoint-692/config.json
Model weights saved in ./xlm-roberta-base-finetuned-squad-4/checkpoint-692/pytorch_model.bin
tokenizer config file saved in ./xlm-roberta-base-finetuned-squad-4/checkpoint-692/tokenizer_config.json
Special tokens file saved in ./xlm-roberta-base-finetuned-squad-4/checkpoint-692/special_tokens_map.json
tokenizer config file saved in ./xlm-roberta-base-finetuned-squad-4/tokenizer_config.json
Special tokens file saved in ./xlm-roberta-base-finetuned-squad-4/special_tokens_map.json
Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.
***** Running Evaluation *****
  Num examples = 2359
  Batch size = 16


  0%|          | 0/2210 [00:00<?, ?it/s]

Saving model checkpoint to ./xlm-roberta-base-finetuned-squad-4/checkpoint-1384
Configuration saved in ./xlm-roberta-base-finetuned-squad-4/checkpoint-1384/config.json
Model weights saved in ./xlm-roberta-base-finetuned-squad-4/checkpoint-1384/pytorch_model.bin
tokenizer config file saved in ./xlm-roberta-base-finetuned-squad-4/checkpoint-1384/tokenizer_config.json
Special tokens file saved in ./xlm-roberta-base-finetuned-squad-4/checkpoint-1384/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2359
  Batch size = 16


In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained("xlm-roberta-base-finetuned-squad_au/checkpoint-4000")
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad_au",
    evaluation_strategy = "epoch",
    logging_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_steps=2,
    metric_for_best_model='f1',
    save_total_limit=4,
    push_to_hub=True,
)
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


loading configuration file xlm-roberta-base-finetuned-squad_au/checkpoint-4000/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base-finetuned-squad_au/checkpoint-4000",
  "architectures": [
    "XLMRobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights file xlm-roberta-base-finetuned-squad_au/checkpoint-4000/pytorch_model.bin
All model checkpoint weights were us

In [ ]:
trainer.push_to_hub(commit_message="Training complete baseline")

Saving model checkpoint to xlm-roberta-base-finetuned-squad_au
Configuration saved in xlm-roberta-base-finetuned-squad_au/config.json
Model weights saved in xlm-roberta-base-finetuned-squad_au/pytorch_model.bin
tokenizer config file saved in xlm-roberta-base-finetuned-squad_au/tokenizer_config.json
Special tokens file saved in xlm-roberta-base-finetuned-squad_au/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.33k/1.03G [00:00<?, ?B/s]

Upload file training_args.bin:  99%|#########9| 3.34k/3.36k [00:00<?, ?B/s]

Upload file runs/Nov21_15-19-13_391905903de9/events.out.tfevents.1669043963.391905903de9.79.0:  64%|######3   …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/chieunq/xlm-roberta-base-finetuned-squad_au
   3aad80f..c46c2b4  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/chieunq/xlm-roberta-base-finetuned-squad_au
   3aad80f..c46c2b4  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Question Answering', 'type': 'question-answering'}}
To https://huggingface.co/chieunq/xlm-roberta-base-finetuned-squad_au
   c46c2b4..58475d8  main -> main

   c46c2b4..58475d8  main -> main



'https://huggingface.co/chieunq/xlm-roberta-base-finetuned-squad_au/commit/c46c2b4af346673e68b62f1533d6e9ebb6a84f05'

We can now finetune our model by just calling the `train` method:

Since this training is particularly long, let's save the model just in case we need to restart.

In [ ]:
trainer.save_model("/content/drive/MyDrive/Colab/UIT_vQuaD/model")

Saving model checkpoint to /content/drive/MyDrive/Colab/UIT_vQuaD/model
Configuration saved in /content/drive/MyDrive/Colab/UIT_vQuaD/model/config.json
Model weights saved in /content/drive/MyDrive/Colab/UIT_vQuaD/model/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Colab/UIT_vQuaD/model/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab/UIT_vQuaD/model/special_tokens_map.json
Saving model checkpoint to xlm-roberta-base-finetuned-squad_1
Configuration saved in xlm-roberta-base-finetuned-squad_1/config.json
Model weights saved in xlm-roberta-base-finetuned-squad_1/pytorch_model.bin
tokenizer config file saved in xlm-roberta-base-finetuned-squad_1/tokenizer_config.json
Special tokens file saved in xlm-roberta-base-finetuned-squad_1/special_tokens_map.json
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Question Answering', 'type': 'question-answering'}}


In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "chieunq/xlm-roberta-base-finetuned-squad-2"
question_answerer = pipeline("question-answering", model=model_checkpoint)

context = """
Nhóm của chúng tôi là sinh viên năm 4 trường ĐH Công Nghệ - ĐHQG Hà Nội. Nhóm gồm 3 thành viên : Nguyễn Quang Chiều, Nguyễn Quang Huy và Nguyễn Trần Anh Đức.\
 Đây là pha Reader trong dự án cuồi kì môn Các vấn đề hiện đại trong CNTT của nhóm . 
"""
question = "3 thành viên trong nhóm gồm những ai ?"
question_answerer(question=question, context=context)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--chieunq--xlm-roberta-base-finetuned-squad-2/snapshots/2ca64af4fbc75716fb417662c29b9ab8e4e015f5/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "chieunq/xlm-roberta-base-finetuned-squad-2",
  "architectures": [
    "XLMRobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading configuration file con

{'score': 0.9928902387619019,
 'start': 98,
 'end': 158,
 'answer': 'Nguyễn Quang Chiều, Nguyễn Quang Huy và Nguyễn Trần Anh Đức.'}